In [17]:
# List of TTPs to check
ttp_list = [
    'T1194', 'T1088', 'T1346', 'T1065', 'T1500', 'T1311', 'T1432', 'T1002', 'T1043', 
    'T1433', 'T1329', 'T1312', 'T1179', 'T1517', 'T1093', 'T0872', 'T1079', 'T1009', 
    'T0840', 'T1099', 'T1053.004', 'T1045', 'T1426', 'T1100', 'T1512', 'T1022', 'T1438', 
    'T1420', 'T1532', 'T1508', 'T0871', 'T1533', 'T1476', 'T1077', 'T1191', 'T1402', 
    'T1341', 'T1024', 'T1117', 'T1047.001', 'T1345', 'T1004', 'T1497.004', 'T1064', 
    'T1247', 'T1089', 'T1249', 'T1086', 'T1260', 'T1061', 'T0853', 'T1143', 'T0847', 
    'T1122', 'T1084', 'T1351', 'T1268', 'T0807', 'T1023', 'T1107', 'T1444', 'T1660', 
    'T0831', 'T0809', 'T1081', 'T1101', 'T1437', 'T1076', 'T1503.004', 'T1035', 'T1412', 
    'T1308', 'T1328', 'T1013', 'T1031', 'T1060', 'T1193', 'T1085', 'T0855', 'T1513', 
    'T1116', 'T1168', 'T1094', 'T1447', 'T1138', 'T1063', 'T1183', 'T1032', 'T1158', 
    'T1418', 'T1502', 'T1337', 'T1145', 'T1575', 'T1347', 'T1503.003', 'T1503', 'T1429', 
    'T1544', 'T1050', 'T1108', 'T1073', 'T1015', 'T1038', 'T1192', 'T1362', 'T1066'
]

In [18]:
# List of STIX file versions with raw URLs for each version
versions = [
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-2.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-3.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-4.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-5.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-6.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-7.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-8.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-9.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-10.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-11.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-12.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-13.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-14.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-15.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-16.0.json"
]


In [19]:
#ttp_list = ['T1194', 'T1077']

In [20]:
import requests

def get_stix_data(url):
    """
    Fetches STIX data for a given MITRE ATT&CK version from a URL.
    
    Args:
        url (str): URL to fetch the STIX data.
    
    Returns:
        dict: Parsed STIX data as JSON.
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error fetching data from {url}. Status code: {response.status_code}")

def find_technique_in_version(stix_data, ttp_id):
    """
    Searches for a technique ID or sub-technique ID in the STIX data.
    
    Args:
        stix_data (dict): STIX data as parsed JSON.
        ttp_id (str): The technique ID or sub-technique ID to search for.
    
    Returns:
        dict or None: Returns the technique entry if found, else None.
    """
    for obj in stix_data.get('objects', []):
        if obj.get('type') == 'attack-pattern':
            # Check if external references contain the exact ttp_id
            for ref in obj.get('external_references', []):
                if ref.get('external_id') == ttp_id:
                    return obj
    return None


def track_techniques_across_versions(ttp_list, versions):
    """
    Tracks the introduction and removal/deprecation of techniques across ATT&CK versions.
    
    Args:
        ttp_list (list): List of technique IDs to track.
        versions (list): List of version URLs to check.
    
    Returns:
        dict: Dictionary with tracking info for each technique.
    """
    technique_info = {}

    for ttp_id in ttp_list:
        first_appearance = None
        deprecated_version = None
        revoked_version = None
        
        for version_url in versions:
            stix_data = get_stix_data(version_url)
            version_name = version_url.split("/")[-1].split(".")[0]
            
            technique = find_technique_in_version(stix_data, ttp_id)
            if technique:
                if not first_appearance:
                    first_appearance = version_name
                
                if technique.get('revoked') and not revoked_version:
                    revoked_version = version_name
                if technique.get('x_mitre_deprecated') and not deprecated_version:
                    deprecated_version = version_name
        
        technique_info[ttp_id] = {
            'first_appearance': first_appearance,
            'deprecated_version': deprecated_version,
            'revoked_version': revoked_version
        }
    
    return technique_info

# Track the techniques across the versions
technique_tracking = track_techniques_across_versions(ttp_list, versions)

# Output the results
for ttp_id, info in technique_tracking.items():
    print(f"TTP ID: {ttp_id}")
    print(f"  First Appearance: {info['first_appearance']}")
    print(f"  Deprecated Version: {info['deprecated_version']}")
    print(f"  Revoked Version: {info['revoked_version']}\n")


TTP ID: T1194
  First Appearance: enterprise-attack-2
  Deprecated Version: None
  Revoked Version: enterprise-attack-7

TTP ID: T1088
  First Appearance: enterprise-attack-1
  Deprecated Version: None
  Revoked Version: enterprise-attack-7

TTP ID: T1346
  First Appearance: None
  Deprecated Version: None
  Revoked Version: None

TTP ID: T1065
  First Appearance: enterprise-attack-1
  Deprecated Version: None
  Revoked Version: enterprise-attack-7

TTP ID: T1500
  First Appearance: enterprise-attack-4
  Deprecated Version: None
  Revoked Version: enterprise-attack-7

TTP ID: T1311
  First Appearance: None
  Deprecated Version: None
  Revoked Version: None

TTP ID: T1432
  First Appearance: None
  Deprecated Version: None
  Revoked Version: None

TTP ID: T1002
  First Appearance: enterprise-attack-1
  Deprecated Version: None
  Revoked Version: enterprise-attack-7

TTP ID: T1043
  First Appearance: enterprise-attack-1
  Deprecated Version: enterprise-attack-7
  Revoked Version: None

T